In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import *
import numpy as np
import random
import torchvision.transforms as transforms
import torch.nn as nn
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import timm
import os

In [2]:
class CustomDataset(Dataset):
    def __init__(self, path, transform):
        self.path = path
        self.transform = transform                
        
    def __len__(self):
        return len(self.path)
    
    def __getitem__(self, idx):
        file = self.path[idx]
        
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        transformed = self.transform(image=image)['image']
        
        return transformed

In [3]:
# transform = transforms.Compose([
#     transforms.Resize((224,224)),
#     transforms.GaussianBlur(7),
#     transforms.RandomHorizontalFlip(p=0.2),
#     transforms.RandomVerticalFlip(p=0.2),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,0.5,0.5),(0.224,0.224,0.224))
# ])


transform = A.Compose([
    A.Resize(224,224),
#     A.GaussNoise(p=0.1),
#     A.HorizontalFlip(p=0.2),
#     A.ColorJitter(),
    A.Normalize(),
    ToTensorV2(),
])
valt=A.Compose([
  A.Resize(224,224),
    A.Normalize(),
    ToTensorV2(),
])

# RandomResizeCrop
# Random GaussianNoise
# color jitter -> b/s/h/c  0.2 ~ 3  ||  128
# RandomHorzital

In [4]:
cnt=0
ctoi={k : idx for idx,k in enumerate(os.listdir('/media/data2/rjsdn/dacon/basic_image_classification/train'))}
itoc = {v: k for k, v in ctoi.items()}
ctoi

{'deer': 0,
 'frog': 1,
 'automobile': 2,
 'dog': 3,
 'truck': 4,
 'bird': 5,
 'cat': 6,
 'ship': 7,
 'horse': 8,
 'airplane': 9}

In [5]:
test_path = '/media/data2/rjsdn/dacon/basic_image_classification/test/'

testset = [test_path+x for x in sorted(os.listdir(test_path))]
test = CustomDataset(testset,transform=valt)


In [6]:
testloader = DataLoader(test,batch_size=128,num_workers=4)

In [7]:
e_num=2
model_name = 'resnet34'

In [ ]:
names=['weights/resnet18_224_123/','weights/Eb1_224_123/']
model_names=['resnet18','efficientnet_b1']

In [ ]:
models = []
for i in range(e_num):
#     model = timm.create_model(model_name,num_classes=10)
    model = timm.create_model(model_names[i],num_classes=10)
#     model.load_state_dict(torch.load(f'weights/resnet34_224_a_{i+1}/best.path.tar')['model_state_dict'])
    model.load_state_dict(torch.load(f'weights/{names[i]}/best.path.tar')['model_state_dict'])
    model.cuda();model.eval()
    models.append(model)

In [ ]:
pred=[]
with torch.no_grad():        
    for data in tqdm(testloader):
        x = data.cuda()
        tmp = None
        soft=0
        for model in models:
            output = model(x)
            if tmp!=None:
                soft = torch.sum(tmp,output)
            else:
                soft = output
        
        output = torch.argmax(soft,1)
        ans = [itoc[x.item()] for x in output]
        pred=np.append(pred,ans)

In [ ]:
import pandas as pd

ss = pd.read_csv('/media/data2/rjsdn/dacon/basic_image_classification/sample_submission.csv')
ss.target=pred
ss.to_csv('sub.csv',index=False)